# Introduction : observer les traitements

Vous connaissez Apache Spark comme le moteur de calcul distribué et puissant qui est largement utilisé pour les traitements batch de données à grande échelle. Cependant, comme tout autre outil de traitement de données de cette envergure, la performance de Spark peut être grandement affectée par la façon dont vous configurez Spark, vous utilisez Spark et la façon dont vous gérez vos données au sein de Spark. Une configuration inadéquate, un mauvais choix d'opérations ou un mauvais choix de clé dans vos datasets peuvent conduire à des performances médiocres (par exemple, par sur-utilisation des disques et du réseau), voire à des échecs d'application (par exemple, en ramenant trop de données sur un nœud Spark, provoquant ainsi un _out of memory error_ ou _OOM_).

Cette formation a pour de comprendre les mécanismes internes de Spark et d'apprendre à percevoir dans vos applications les parties qui pourraient être améliorées afin de gagner en temps de calcul.

Dans ce notebook, nous allons commencer par mieux appréhender les outils de monitoring proposés par Spark (typiquement, Spark UI) et voir comment ces outils réagissent face à des opérations Spark SQL simples. Nous allons donc particulièrement nous intéresser aux plans d'exécution issue de différents types de requête.

## Prélude

Nous utilisons ici le moteur de notebook [Jupyter](https://jupyter.org/). Celui-ci a été développé en Python et fait partie des moteurs de notebook parmi les plus utilisés du moment (avec Databricks notebook, Zeppelin et Polynote).

Jupyter se base sur des _kernels_ afin de faire fonctionner divers langages dans ses notebooks. Nous nous basons ici sur le _kernel_ [Almond](https://almond.sh/), qui utilise l'interpréteur Scala [Ammonite](https://ammonite.io/).

👷 Exécuter les deux cellules suivantes.

In [ ]:
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`
import $ivy.`org.apache.logging.log4j:log4j-api:2.8.2`
import $ivy.`org.apache.logging.log4j:log4j-slf4j-impl:2.8.2`

// Avoid disturbing logs
import org.apache.log4j._
import org.apache.log4j.varia._
BasicConfigurator.configure(NullAppender.getNullAppender())

import $ivy.`org.apache.spark::spark-core:3.5.0`
import $ivy.`org.apache.spark::spark-sql:3.5.0`

In [ ]:
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val spark = {
  NotebookSparkSession.builder()
    .progress(enable = true, keep = true, useBars = false)
    .master("local[*]")
    .appName("Spark tuning – Introduction")
    .getOrCreate()
}

import spark.implicits._
val sparkContext = spark.sparkContext
import $file.^.internal.spark_helper
import spark_helper.implicits._

## Analyse d'un chargement de fichier CSV

Nous allons commencer par charger un fichier CSV et observer ce qu'il se passe dans Spark UI.

In [ ]:
%%data limit=10

val orders: DataFrame =
  spark.read
    .option("header", true)
    .csv("data/orders.csv")

orders.printSchema()
orders

👀 **Ce qu'il faut voir** 👀

Avant de basculer sur Spark UI, Spark (via Almond) fait apparaître deux tâches (done), avec un libellé du style `csv at cell...` ou `showHTML`. Ces tâches représentent en fait des jobs Spark.

Ici, le premier job permet de récupérer le nom des colonnes du contenu du fichier CSV. En effet, nous n'avons rien précisé concernant la structure du fichier CSV à part qu'il contient un en-tête. Spark doit donc récupérer cet en-tête du fichier, afin de déterminer le nom des colonnes.

Le second job est lancé par la commande `.showHTML` qui ajouté sur la dernière ligne de la cellule par `%%data`.

## Spark UI

* Allez dans Spark UI (`http://<hostname>:4040`)
* Cliquez sur l'onglet "SQL"

À ce niveau, vous allez voir 2 requêtes. La première (ID=0) est nommée "csv..." et la seconde (ID=1) est nommée "show...".
    
🤔 **Question** 🤔

 * Pour la requête 0, combien de lignes ont été extraites du fichier `orders.csv` ? Expliquez ce nombre.
 * Pour la requête 1, combien de lignes ont été extraites du fichier `orders.csv` ? Expliquez ce nombre.

Note : le fichier contient environ 10 000 lignes.

### Plan d'exécution
    
Dans Spark UI, affichez le détail de la requête 1. En bas, de la page, cliquez sur `> Details`.
    
Dans l'encadré "Details", vous voyez apparaître les plans d'exécution de votre requête. 4 plans sont affichés :
    
* "Parsed Logical Plan" : c'est le plan d'exécution qui est issue directement de votre requête.
* "Analyzed Logical Plan" : c'est à nouveau le plan de votre requête, mais avec la résolution des noms et des types.
* "Optimized Logical Plan" : il s'agit d'une première optimisation de votre requête, sans considérer le support physique.
* "Physical Plan" : il s'agit du plan qui sera effectivement exécuté. Il a été généré après une seconde optimisation prenant en compte le support physique.
    
Chacun des plans se lisent du bas vers le haut : la première opération du traitement se trouve en bas et la dernière opération du traitement se trouve en haut.

🤔 **Question** 🤔

Dans le plan physique, au niveau de l'opération,

* Quel type d'opération est lancée en premier lieu ?
* Remarquez la structure qui sera extraite de ce traitement.

### Explain

Une façon de voir le plan d'exécution, c'est d'utiliser la méthode `.explain()`.

In [ ]:
orders.explain()

Il est possible d'utiliser ces options sous forme de chaîne de caractères sur la méthode `.explain()`

* simple affiche uniquement le plan physique.
* extended: affiche le plan logique et le plan physique.
* codegen: affiche le plan physique et le code généré s'il est disponible.
* cost: affiche le plan logique et les statistiques, si elles sont disponibles.
* formatted: sépare la sortie en deux parties : une vue générale sur le plan physique et les détails de chacun des nœuds du plan.

In [ ]:
orders.explain("formatted")

### Explain ou Spark UI pour le plan d'exécution ?

La méthode `.explain()` permet d'obtenir immédiatement dans un code diverses informations concernant une requête, sans avoir à rechercher l'URL de Spark UI. Cependant, `.explain()` a une limitation : `.explain()` vous fournira les informations qu'il pourra obtenir d'une requête avant son exécution. De fait, `.explain()` ne donnera pas le plan qui a été effectivement exécuté, en particulier, les temps intermédiaires d'exécution ou les dernières optimisations qui ont été apportées sur le plan physique, suite aux statistiques obtenues sur les données, ou les modifications liées au cache.

De son côté, Spark UI offre la possibilité, une fois la requête exécutée, de voir
* une représentation graphique du plan d'exécution (sous forme de DAG),
* le plan physique optimisé,
* le rattachement à des jobs Spark Core et les phases d'échange de données,
* l'ensemble des caches utilisés.

Par contre, sur de la données volumineuse, il faut attendre que le traitement soit fait pour avoir le plan d'exéction final avec les stats.

Donc, `.explain()` est donc bien pour avoir rapidement une idée de ce que va faire une requête. Spark UI est bien pour voir dans le détail ce qu'il s'est réellement passé.

## Projection

👷 Lancez la requête ci-dessous, qui effectue une projection des données sur une colonne, en utilisant `.select()`.

In [ ]:
%%data limit=10

orders
  .select($"product")

🤔 **Question** 🤔

Dans le plan d'exécution associé à la requête ci-dessus,

* Quelle structure de donnée est récupérée ?
* Est-elle différente de celle extraite par la requête précédente ?

## Filtre

👷 Exécutez la requête ci-dessous, qui filtre les données avec `.where().

In [ ]:
%%data limit=10

orders
  .where($"product" === "expresso")

🤔 **Question** 🤔

Selon le plan physique de cette requête,

* Quel est le filtre qui est appliqué sur les données ?
* Combien de fois ce filtre apparaît dans le plan physique ?

### Pushed down filters

Lors de la lecture du fichier, nous pouvons voir que le filtre est aussi appliqué. C'est ce qu'indique le champ `PushedFilters`. Cette fonctionnalité est appelée _pushed down filters_. Elle consiste à appliquer les filtres que vous utilisez dans votre requête (haut niveau) au niveau du support physique (niveau bas). Cette fonctionnalité est surtout intéressante pour des sources de données comme les formats orientés colonnes (Parquet, ORC) ou les connexions JDBC, etc. Toutes les sources permettant de filtrer nativement les données.

### WholeStageCodeGen

Si vous regardez le DAG de la requête, vous voyez que l'opération Filter est inclu dans un nœud plus vaste, nommé `WholeStageCodegen`. Ce nœud indique que l'opération Filter a été convertie en code Java et qu'elle a été compilée au runtime avant d'être exécutée.

Le `WholeStageCodegen` présente ces avantages :
* Permettre de regrouper une série d'opérations en une seule par composition.
* Utiliser implicitement les espaces mémoires off-heap, qui ne subit pas l'intervention du GC et dont les accès sont optimisés.

## Deux opérations en une

👷 Exécutez la cellule ci-dessous et allez voir comment est interprétée cette requête dans Spark UI.

In [ ]:
%%data limit=10

orders
  .where($"product" === "expresso")
  .select($"client", $"price" * 2)

🤔 **Question** 🤔

Est-ce que Spark a regroupé les 2 opérations de la requête ci-dessus en un seul bloque ?

### Afficher le code généré

Il est possible d'afficher le code généré par Spark.

👷 Exécuter la cellule ci-dessous et allez voir au niveau de la ligne 71 dans le code généré.

In [ ]:
orders
  .where($"product" === "expresso")
  .select($"client", $"price" * 2)
  .explain("codegen")

## Agrégation

Nous avons vu jusque-là des opérations linéaires (filter, select). Nous allons maintenant voir une opération plus complexe : l'agrégation.

L'agrégation consiste à composer une collection de valeur afin d'obtenir une valeur résultante unique. Typiquement, il peut s'agir de compter le nombre d'éléments dans cette collection, de faire la somme des valeurs contenues s'il s'agit de nombres, de calculer un prix total sur une commande, de déterminer le dernier événement à prendre en compte...

Dans le cadre de Spark, la collection considérée est un dataframe. Le problème auquel nous faisons face avec Spark, c'est qu'un dataframe s'apparente à une collection dont les éléments sont éparpillés sur plusieurs machines, et donc executors. Pour effectuer une agrégation dans ce cas, il faut commencer par réaliser cette agrégation en local sur chaque executor, afin d'obtenir un résultat intermédiaire. Les différents résultats intermédiaires sont ensuite envoyés au driver qui terminera l'agrégation.

👷 Nous allons voir ce que cela donne avec un simple `.count`.

In [ ]:
orders.count

Sur une agrégation de ce type, le plan d'exécution ne peut être vu que dans Spark UI.

Dans Spark UI, nous pouvons remarquer que le plan physique est divisé en deux sous-plan :
* "Initial Plan" : plan physique initialement calculé par Spark
* "Final Plan" : il s'agit d'une révision du plan physique initial, après avoir pris en compte le volume des données.

Dans le plan physique, nous voyons apparaître `AdaptiveSparkPlan isFinalPlan=true`. Ce qui signifie que lors de l'exécution de la requête, Spark a calculé d'après les volumétries à traiter qu'une optimisation était encore possible et qu'il l'a appliquée. Il s'agit de la mise en pratique de la fonctionnalité _Adaptive Query Execution_ (ou AQE), qui est une nouveauté de Spark 3.

Dans le plan physique, nous pouvons voir les opérations suivantes :
* `HashAggregate` : indique qu'une agrégation de données est effectuée par rapport aux clés indiquées dans les paramètres de l'opération, dont un hash est calculé. Ici, nous pouvons observer qu'aucune clé n'est utilisée. Dans les paramètres, nous pouvons observer aussi `partial_count(1)` pour la première agrégation (il s'agit d'une agrégation locale) et `count(1)` pour la seconde agrégation (il s'agit de l'agrégation finale).
* `Exchange SinglePartition` : `Exchange` signifie que des données sont échangées entre plusieurs executors. `SinglePartition` indique que les données sont dans une seule partition. Il y a donc un échange de données dans une seule partition. Autrement dit, l'opération ne fait rien.
* `ShuffleQueryStage` : il s'agit de la phase d'échange proprement. Cette opération est ajoutée aussi par la fonctionnalité AQE afin de permettre de dissocier les opérations aval des autres opérations et de permettre à Spark de changer de stratégie en fonction des volumétries observées.

#### Association avec Spark Core

👷 Dans le haut de la page Spark UI, montrant les détails de la requête, vous avez un champ indiquant les jobs associés en succès. Il y en a deux. Cliquez sur celui le plus à droite.

Dans la page qui vient de s'ouvrir, vous voyez apparaître le DAG vu par Spark Core. Il est divisé en deux parties. Chaque partie correspond à un _stage_. Un _stage_ est une suite d'opération Spark Core dans laquelle toutes les opérations se font en local dans l'executor. Dès qu'on sort d'un stage pour entrer dans un autre, il y a un échange de données qui s'effectue entre les executors.

L'objectif est bien sûr d'avoir le moins de stage possible dans un traitement. En effet, un échange de données signifie une utilisation du réseau et une baisse des performances.

Dans le DAG affiché dans Spark UI, nous pouvons voir que le stage de gauche est grisé et annoté _(skipped)_. C'est parce que le premier stage est en réalité effectué par le premier job de notre requête.



### Moyenne

Voici une autre requête effectuant une agrégation au moyen cette fois de `.groupBy()` et de `.agg()`.

La particularité ici par rapport à `.count`, c'est qu'à travers la méthode `.groupBy()`, nous déterminons une clé pour regrouper les valeurs. Ceci signifie que les données vont être redistribuées, afin que celles ayant la même clé se retrouvent au niveau du même executor. Une fois la redistribution faite, l'agrégation par clé peut se faire.

In [ ]:
%%data limit=10

import org.apache.spark.sql.functions._

orders
  .groupBy($"client")
  .agg(avg($"price") as "avg_price")

🤔 **Question** 🤔

* Quelles différences voyez-vous avec le plan d'exécution dans le cas du `.count` ?

## Jointure

Nous allons ce que donne le plan d'exécution dans le cadre d'une jointure.

Pour cela, nous allons utiliser un fichier de correspondance entre les identifiants des clients et leur nom.

In [ ]:
%%data limit=10

val mapping = spark.read.option("header", true).csv("data/client-mapping.csv")

mapping.printSchema()
mapping

Le code ci-dessous effectue une jointure entre nos deux dataframes.

La jointure est une opération particulière qui implique de redistribuer sur le cluster les données, un peu comme pour l'agrégation. Mais pour la jointure, il existe divers stratégie que nous étudierons dans un prochain Notebook.

In [ ]:
%%data limit=10

orders
  .join(mapping, $"client" === $"clientId")

🤔 **Question** 🤔
* Comment est représentée la jointure dans le graphe dans Spark UI ?

La notion de _broadcast_ implique l'idée de partir d'une valeur et de la copier telle quelle au niveau de l'ensemble des exécuteurs.

## User-Defined Function (UDF)

Nous allons voir maintenant l'effet d'une UDF sur le plan d'exécution.

👷 Exécutez la cellule ci-dessous qui utilise les fonctions _builtins_ de Spark.

In [ ]:
%%data limit=10

val q1 = orders.withColumn("date", to_date($"timestamp"))

q1

👷 Exécutez la cellule ci-dessous qui définie et utilise une UDF.

In [ ]:
%%data limit=10

import org.apache.spark.sql.types._

def toDate(timestamp: java.sql.Timestamp): java.sql.Date = {
  java.sql.Date.valueOf(timestamp.toLocalDateTime().toLocalDate())
}

val toDate_udf = udf(toDate(_)).withName("toDate_udf")

val q2 = orders.withColumn("date", toDate_udf($"timestamp"))

q2

🤔 **Question** 🤔

Selon le plan d'exécution de ces deux requêtes, quels différences apparaissent ?

In [ ]:
q1.explain("formatted")

In [ ]:
q2.explain("codegen")